In [2]:
# Importemos librerias y datos
import numpy as np
import pandas as pd

# dataset
df = pd.read_csv("https://github.com/robintux/Datasets4StackOverFlowQuestions/raw/master/Historical_Product_Demand.csv.zip")

# Numero de filas y columnas
print(df.shape)

# Nombres de columnas
print(df.columns)

# Tipo de datos para cada columna
print(df.dtypes)

(1048575, 5)
Index(['Product_Code', 'Warehouse', 'Product_Category', 'Date',
       'Order_Demand'],
      dtype='object')
Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object


# Limpieza de datos


In [3]:
# Valores faltantes
df.isna().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

In [4]:
# Valores nulos
df.isnull().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

In [5]:
# Eliminemos los valores faltantes de nuestro dataframe
df.dropna(inplace= True)

In [6]:
df.isna().sum()

Product_Code        0
Warehouse           0
Product_Category    0
Date                0
Order_Demand        0
dtype: int64

In [7]:
# Verifiquemos que la columns Product_Code no sea una variable categorica
df.Product_Code.unique().shape

(2160,)

In [8]:
# Posibles valores de la columna Warehouse
df.Warehouse.unique()

array(['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A'], dtype=object)

In [9]:
# Posibles valores de la columna Product_Category
df.Product_Category.nunique()

33

In [10]:
# Columna Date : Veamos datos de que años tenemos
# Transformamos la columna Date a un objeto datetime
np.sort(pd.to_datetime(df.Date).dt.year.unique())

array([2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype=int32)

In [11]:
# Posibles valores de la columna Order_Demand
df.Order_Demand.unique()

array(['100 ', '500 ', '50000 ', ..., '(3750)', '(191)', '28250 '],
      dtype=object)

In [12]:
df.Order_Demand

0           100 
1           500 
2           500 
3           500 
4           500 
           ...  
1048570    1000 
1048571       1 
1048572    2500 
1048573      50 
1048574       4 
Name: Order_Demand, Length: 1037336, dtype: object

In [15]:
# Plan de accion para la columna Order_Demand

# Primer Paso : Conocer que caracteres conformar (usa) esa columna Order_Demand
# Segunda paso : Usar un expresion regular (regex) para eliminar esos caracteres no numericos


In [13]:
# Primer Paso + Medir tiempo de procesamiento
import time
inicio1 = time.time()
Caracteres1 =  df.applymap(list).Order_Demand.apply(pd.Series).stack().unique()
fin1 = time.time()

print("""
Caracteres :
  %s

(Tiempo de procesamiento : %d)
""" %(Caracteres1 , fin1-inicio1))

/tmp/ipykernel_33/2918209909.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Caracteres1 =  df.applymap(list).Order_Demand.apply(pd.Series).stack().unique()



Caracteres :
  ['1' '0' ' ' '5' '4' '6' '2' '3' '8' '9' '7' '(' ')']

(Tiempo de procesamiento : 207)



In [14]:
# Otra forma de hacer lo mismo si sabemos que caracteres deberian de aparecer
# en la columna Order_Demand (speedup)

inicio2 = time.time()
Caracteres2 = set(list(df["Order_Demand"].str.strip("0123456789").sum()))
fin2 = time.time()

print("""
Caracteres :
  %s

(Tiempo de procesamiento : %d)
""" %(Caracteres2 , fin2-inicio2))


Caracteres :
  {'9', ' ', '8', '4', '0', '3', '2', ')', '5', '6', '7', '1', '('}

(Tiempo de procesamiento : 28)



In [15]:
# Eliminemos estos caracteres no numericos : " " "(" ")"
df["Order_Demand"] = df["Order_Demand"].replace('[\$,)]', "", regex= True).replace('[(]', "", regex = True).astype(int)

In [17]:
# INformacion de la columnas Date
df.Date

0          2012/7/27
1          2012/1/19
2           2012/2/3
3           2012/2/9
4           2012/3/2
             ...    
1048570    2016/4/27
1048571    2016/4/27
1048572    2016/4/28
1048573    2016/10/7
1048574    2016/6/27
Name: Date, Length: 1037336, dtype: object

In [18]:
# Transformamos la columna Date a un dato de tipo datetime
# Construimos variables de interes :
df["Date"] = pd.to_datetime(df["Date"], format = "%Y/%m/%d")
df["Dayofweek"] = df.Date.dt.dayofweek
df["Month"] = df.Date.dt.month
df["Year"] = df.Date.dt.year
df["Period"] = df.Date.dt.to_period("M")

# Ordenemos el dataframe por intermedio de la columna Date
df.sort_values("Date", inplace = True)


In [19]:
df

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Dayofweek,Month,Year,Period
44799,Product_0965,Whse_A,Category_006,2011-01-08,2,5,1,2011,2011-01
72252,Product_1724,Whse_A,Category_003,2011-05-31,108,1,5,2011,2011-05
8431,Product_1521,Whse_S,Category_019,2011-06-24,85000,4,6,2011,2011-06
8432,Product_1521,Whse_S,Category_019,2011-06-24,7000,4,6,2011,2011-06
72669,Product_1507,Whse_C,Category_019,2011-09-02,1250,4,9,2011,2011-09
...,...,...,...,...,...,...,...,...,...
1009604,Product_1710,Whse_J,Category_026,2017-01-06,15,4,1,2017,2017-01
921328,Product_1359,Whse_J,Category_019,2017-01-06,100000,4,1,2017,2017-01
908447,Product_0250,Whse_C,Category_007,2017-01-09,148,0,1,2017,2017-01
863020,Product_1904,Whse_C,Category_019,2017-01-09,4000,0,1,2017,2017-01


In [20]:
# MOdificaciones extras

# Agreguemos un 5% de valores Nan a Order_Demand y para todas las filas de mayo del 2012
df.loc[df.sample(frac = 0.05).index, ["Order_Demand"]] = np.nan
df.loc[(df["Period"] == "2012-05"), "Order_Demand"] = np.nan

# Agreguemos una variable ficticia : Usuario al azar
df["User"] = np.random.choice(["User1", "User2"], size = df.shape[0])

# Pivot_Table

In [21]:
# Calculemos el numero filas para cada año : usemos el metodo groupby de un objeto de tipo dataframe
df.groupby(["Year"]).size()
# pandas.core.series.Series


Year
2011       640
2012    203635
2013    218298
2014    216404
2015    209661
2016    188645
2017        53
dtype: int64

In [22]:
# Usemos pivot_table de pandas
# Para ellos vamos a establecer los dos parametros que van a permitir
# calcular el numero de filas/observaciones por año
  # index : Year
  # Funcion de agregacion (aggfunc) : size

pd.pivot_table(df,
               index = ["Year"],
               aggfunc = ["size"])

# Dato de tipo Dataframe

,size
Year,
2011,640
2012,203635
2013,218298
2014,216404
2015,209661
2016,188645
2017,53


In [23]:
# Numero de filas/observaciones/registros que existen por mes para cada año donde
# tenemos informacion
# groupbt se realizara en dos niveles :
  # primer nivel : Año
  # segundo nivel : mes

df.groupby(["Year", "Month"]).size().to_frame()

0
Year Month       
2011 1          1
     5          1
     6          2
     9          4
     10         3
...           ...
2016 9      15376
     10     16191
     11     17335
     12     15036
2017 1         53

[68 rows x 1 columns]

In [24]:
# Repliquemos usando pivot_table
pd.pivot_table(df,
               index = ["Year"],
               columns = ["Month"],
               aggfunc = ["size"])

size                                                                 \
Month       1        2        3        4        5        6        7        8    
Year                                                                            
2011       1.0      NaN      NaN      NaN      1.0      2.0      NaN      NaN   
2012   15614.0  18123.0  18604.0  16590.0  17014.0  16911.0  17718.0  16492.0   
2013   16638.0  17119.0  17397.0  17685.0  17786.0  16421.0  19085.0  16660.0   
2014   18013.0  18214.0  19839.0  18077.0  16639.0  17762.0  18867.0  16021.0   
2015   18245.0  18116.0  19122.0  17603.0  15572.0  18413.0  19127.0  15507.0   
2016   14515.0  16130.0  17282.0  15223.0  14487.0  16418.0  15319.0  15333.0   
2017      53.0      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

                                           
Month       9        10       11       12  
Year                                       
2011       4.0      3.0     31.0    598.0  
2012   15613.0  18515.0  17416.0  15025.0  
2013   18946.0  24546.0  18617.0  17398.0  
2014   18970.0  19579.0  17486.0  16937.0  
2015   16656.0  17785.0  17335.0  16180.0  
2016   15376.0  16191.0  17335.0  15036.0  
2017       NaN      NaN      NaN      NaN

In [37]:
# A partir del resultado anterior eliminemos la informacion de los años 2011 y 2017
df  = df[~df["Year"].isin([2011, 2017])]

In [25]:
# El objetivo de trabajar con este dataset es pronosticar la demanda de cada producto
# Vamos a construir un DataFrame multiIndex :
  # Primer Nivel : Producto_Code
  # Segundo Nivel : Year
pd.pivot_table(df,
               index = ["Product_Code", "Year"],
               aggfunc = ["size"])

size
Product_Code Year      
Product_0001 2011     2
             2012   139
             2013   117
             2014   129
             2015   107
...                 ...
Product_2172 2012     6
             2013     5
             2014     5
             2015    10
             2016    23

[10687 rows x 1 columns]

In [26]:
# Agreguemos un tercer nivel al DataFrame MultiIindex anterior
pd.pivot_table(df,
               index = ["Product_Code", "Year", "Month"],
               aggfunc = ["size"])

size
Product_Code Year Month      
Product_0001 2011 12        2
             2012 1        15
                  2        13
                  3        13
                  4         7
...                       ...
Product_2172 2016 7         3
                  8         2
                  9         3
                  10        2
                  11        2

[105753 rows x 1 columns]

In [27]:
# Es mas sencillo utilizar la columna Period
pd.pivot_table(df,
               index = ["Product_Code", "Period"],
               aggfunc = ["size"])

size
Product_Code Period       
Product_0001 2011-12     2
             2012-01    15
             2012-02    13
             2012-03    13
             2012-04     7
...                    ...
Product_2172 2016-07     3
             2016-08     2
             2016-09     3
             2016-10     2
             2016-11     2

[105753 rows x 1 columns]

In [28]:
# Al datamframe multiIndex anterior, agreguemosle la informacion de donde se hizo la venta
pd.pivot_table(df,
               index = ["Product_Code", "Period"],
               columns = ["Warehouse"],
               aggfunc = ["size"])

size                     
Warehouse            Whse_A Whse_C Whse_J Whse_S
Product_Code Period                             
Product_0001 2011-12    NaN    NaN    2.0    NaN
             2012-01    7.0    NaN    8.0    NaN
             2012-02    5.0    NaN    8.0    NaN
             2012-03    6.0    NaN    7.0    NaN
             2012-04    2.0    NaN    5.0    NaN
...                     ...    ...    ...    ...
Product_2172 2016-07    NaN    NaN    3.0    NaN
             2016-08    NaN    NaN    2.0    NaN
             2016-09    NaN    NaN    3.0    NaN
             2016-10    NaN    NaN    2.0    NaN
             2016-11    NaN    NaN    2.0    NaN

[105753 rows x 4 columns]

In [29]:
# Rellenosmos los NaN del dataframe anterior
pd.pivot_table(df,
               index = ["Product_Code", "Period"],
               columns = ["Warehouse"],
               aggfunc = ["size"],
               fill_value = 0)

size                     
Warehouse            Whse_A Whse_C Whse_J Whse_S
Product_Code Period                             
Product_0001 2011-12      0      0      2      0
             2012-01      7      0      8      0
             2012-02      5      0      8      0
             2012-03      6      0      7      0
             2012-04      2      0      5      0
...                     ...    ...    ...    ...
Product_2172 2016-07      0      0      3      0
             2016-08      0      0      2      0
             2016-09      0      0      3      0
             2016-10      0      0      2      0
             2016-11      0      0      2      0

[105753 rows x 4 columns]

In [30]:
# Queremos contar el numero de valores sobre la columna Order_Demand y la columna
# Dayofweek (Argumento : values)
pd.pivot_table(df,
               index = ["Product_Code", "Period"],
               columns = ["Warehouse"],
               values = ["Order_Demand", "Dayofweek"],
               aggfunc = ["count"],
               fill_value = 0)

count                                           \
                     Dayofweek                      Order_Demand          
Warehouse               Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C   
Product_Code Period                                                       
Product_0001 2011-12         0      0      2      0            0      0   
             2012-01         7      0      8      0            6      0   
             2012-02         5      0      8      0            5      0   
             2012-03         6      0      7      0            6      0   
             2012-04         2      0      5      0            2      0   
...                        ...    ...    ...    ...          ...    ...   
Product_2172 2016-07         0      0      3      0            0      0   
             2016-08         0      0      2      0            0      0   
             2016-09         0      0      3      0            0      0   
             2016-10         0      0      2      0            0      0   
             2016-11         0      0      2      0            0      0   

                                    
                                    
Warehouse            Whse_J Whse_S  
Product_Code Period                 
Product_0001 2011-12      2      0  
             2012-01      8      0  
             2012-02      6      0  
             2012-03      7      0  
             2012-04      5      0  
...                     ...    ...  
Product_2172 2016-07      3      0  
             2016-08      2      0  
             2016-09      3      0  
             2016-10      2      0  
             2016-11      2      0  

[105753 rows x 8 columns]

In [ ]:
pd.pivot_table(df,
               index = ["Product_Code", "Period"],
               columns = ["Order_Demand"],
               aggfunc = ["count", "size"],
               fill_value = 0)

# Se acabo la memoria RAM